In [2]:
import json
import math
import requests

import modin.pandas as mpd
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio

In [3]:
pio.renderers.default = "iframe"

In [4]:
def get_nytimes_state_data(url=None):
    """
    Returns a DataFrame of state-level Covid-19 data from the New York Times' GitHub.
    
    Args:
        url (str): URL to csv file.
        
    Returns:
        pd.DataFrame
    """
    if url is None:
        url = r"https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv"
    
    # Read data & pad FIPS
    df = pd.read_csv(url)
    df["fips"] = df["fips"].astype(int).astype(str).str.zfill(2)
    df.rename(columns={"cases": "cum_cases", "deaths": "cum_deaths"}, inplace=True)
    
    # Make tidy
    all_dates = pd.date_range(df["date"].min(), df["date"].max()).astype(str).tolist()
    frames = []
    for state in df["state"].unique():
        df_state = df[df["state"] == state].copy()
        dates_to_add = list(set(all_dates).difference(set(df_state["date"])))
        df_to_add = pd.DataFrame({"date": dates_to_add})
        df_to_add["state"] = state
        df_to_add["fips"] = df_state["fips"].iloc[0]
        # Aggregate
        df_state_new = pd.concat([df_state, df_to_add])
        df_state_new.sort_values(by="date", inplace=True)
        for col in ["cases", "deaths"]:
            df_state_new["cum_{}".format(col)].fillna(method="ffill", inplace=True)
            df_state_new["cum_{}".format(col)].fillna(0, inplace=True)
            df_state_new[col] = df_state_new["cum_{}".format(col)].diff().fillna(0)
        frames.append(df_state_new)
        
    df = pd.concat(frames, ignore_index=True)
    df.sort_values(by=["state", "date"], inplace=True)
    
    return df

In [5]:
def get_nytimes_county_data(url=None):
    """
    Returns a DataFrame of county-level Covid-19 data from the New York Times' GitHub.
    
    Args:
        url (str): URL to csv file.
        
    Returns:
        pd.DataFrame
    """
    if url is None:
        url = r"https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
    
    # Fill in missing FIPS & remove cases in unknown counties
    df = pd.read_csv(url)
    df.loc[df["county"] == "New York City", "fips"] = 36061
    df.loc[df["county"] == "Kansas City", "fips"] = 20085
    df = df[df["county"] != "Unknown"].copy()
    df["fips"] = df["fips"].astype(int).astype(str).str.zfill(5)
    df.rename(columns={"cases": "cum_cases", "deaths": "cum_deaths"}, inplace=True)
    
    # Make tidy
    all_dates = pd.date_range(df["date"].min(), df["date"].max()).astype(str).tolist()
    frames = []
    for fips in df["fips"].unique():
        df_county = df[df["fips"] == fips].copy()
        dates_to_add = list(set(all_dates).difference(set(df_county["date"])))
        df_to_add = pd.DataFrame({"date": dates_to_add})
        df_to_add["county"] = df_county["county"].iloc[0]
        df_to_add["state"] = df_county["state"].iloc[0]
        df_to_add["fips"] = fips
        # Aggregate
        df_county_new = pd.concat([df_county, df_to_add])
        df_county_new.sort_values(by="date", inplace=True)
        for col in ["cases", "deaths"]:
            df_county_new["cum_{}".format(col)].fillna(method="ffill", inplace=True)
            df_county_new["cum_{}".format(col)].fillna(0, inplace=True)
            df_county_new[col] = df_county_new["cum_{}".format(col)].diff().fillna(0)
        frames.append(df_county_new)
            
    df = pd.concat(frames, ignore_index=True)
    df.sort_values(by=["county", "date"], inplace=True)
    
    return df

In [6]:
def modin_get_nytimes_county_data(url=None):
    """
    Returns a DataFrame of county-level Covid-19 data from the New York Times' GitHub.
    
    Args:
        url (str): URL to csv file.
        
    Returns:
        mpd.DataFrame
    """
    if url is None:
        url = r"https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
    
    # Fill in missing FIPS & remove cases in unknown counties
    df = mpd.read_csv(url)
    df.loc[df["county"] == "New York City", "fips"] = 36061
    df.loc[df["county"] == "Kansas City", "fips"] = 20085
    df = df[df["county"] != "Unknown"].copy()
    df["fips"] = df["fips"].astype(int).astype(str).str.zfill(5)
    df.rename(columns={"cases": "cum_cases", "deaths": "cum_deaths"}, inplace=True)
    
    # Make tidy
    all_dates = mpd.date_range(df["date"].min(), df["date"].max()).astype(str).tolist()
    frames = []
    for fips in df["fips"].unique():
        df_county = df[df["fips"] == fips].copy()
        dates_to_add = list(set(all_dates).difference(set(df_county["date"])))
        df_to_add = mpd.DataFrame({"date": dates_to_add})
        df_to_add["county"] = df_county["county"].iloc[0]
        df_to_add["state"] = df_county["state"].iloc[0]
        df_to_add["fips"] = fips
        # Aggregate
        df_county_new = mpd.concat([df_county, df_to_add])
        df_county_new.sort_values(by="date", inplace=True)
        for col in ["cases", "deaths"]:
            df_county_new["cum_{}".format(col)].fillna(method="ffill", inplace=True)
            df_county_new["cum_{}".format(col)].fillna(0, inplace=True)
            df_county_new[col] = df_county_new["cum_{}".format(col)].diff().fillna(0)
        frames.append(df_county_new)
            
    df = mpd.concat(frames, ignore_index=True)
    df.sort_values(by=["county", "date"], inplace=True)
    
    return df

In [7]:
%timeit get_nytimes_county_data()

52.8 s ± 2.77 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit modin_get_nytimes_county_data()

/Users/thomasslebir/opt/anaconda3/envs/covid_viz_env/lib/python3.8/site-packages/modin/error_message.py:45: UserWarning:

Parameters provided defaulting to pandas implementation.
To request implementation, send an email to feature_requests@modin.org.

/Users/thomasslebir/opt/anaconda3/envs/covid_viz_env/lib/python3.8/site-packages/modin/error_message.py:45: UserWarning:

`Series.unique` defaulting to pandas implementation.

/Users/thomasslebir/opt/anaconda3/envs/covid_viz_env/lib/python3.8/site-packages/modin/pandas/dataframe.py:109: UserWarning:

Distributing <class 'dict'> object. This may take some time.

/Users/thomasslebir/opt/anaconda3/envs/covid_viz_env/lib/python3.8/site-packages/modin/error_message.py:45: UserWarning:

`DataFrame.insert` for empty DataFrame defaulting to pandas implementation.

/Users/thomasslebir/opt/anaconda3/envs/covid_viz_env/lib/python3.8/site-packages/modin/pandas/dataframe.py:109: UserWarning:

Distributing <class 'pandas.core.frame.DataFrame'> object. 

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [5]:
url = r"https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"

# Fill in missing FIPS & remove cases in unknown counties
df = pd.read_csv(url)
df.loc[df["county"] == "New York City", "fips"] = 36061
df.loc[df["county"] == "Kansas City", "fips"] = 20085
df = df[df["county"] != "Unknown"].copy()
df["fips"] = df["fips"].astype(int).astype(str).str.zfill(5)
df.rename(columns={"cases": "cum_cases", "deaths": "cum_deaths"}, inplace=True)

# Make tidy
all_dates = pd.date_range(df["date"].min(), df["date"].max()).astype(str).tolist()
fips = df["fips"].unique()[0]

In [10]:
%%timeit
df_filter = df[df["fips"] == fips].copy()
df_filter

7.35 ms ± 78.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit
df_groupby = df.groupby("fips", as_index=False).filter(lambda x: x.name == fips)
df_groupby

396 ms ± 43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%timeit get_nytimes_county_data()

36.5 s ± 1.89 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%load_ext line_profiler

In [15]:
%lprun -f get_nytimes_county_data get_nytimes_county_data()

Timer unit: 1e-06 s

Total time: 37.8782 s
File: <ipython-input-11-a1bc3cc4cd2e>
Function: get_nytimes_county_data at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def get_nytimes_county_data(url=None):
     2                                               """
     3                                               Returns a DataFrame of county-level Covid-19 data from the New York Times' GitHub.
     4                                               
     5                                               Args:
     6                                                   url (str): URL to csv file.
     7                                                   
     8                                               Returns:
     9                                                   pd.DataFrame
    10                                               """
    11         1          3.0      3.0      0.0      if url is None:
    12         1

In [16]:
type(df["fips"].unique())

numpy.ndarray

In [ ]:
def get_usa_state_codes(url=None, table_index=0):
    """
    Returns a DataFrame of United States alpha and FIPS codes.
    
    Args:
        url (str): URL to webpage.
        table_index (int): Position of the table of interest within the page's HTML elements.
        
    Returns:
        pd.DataFrame
    """
    if url is None:
        url = r"https://www.nrcs.usda.gov/wps/portal/nrcs/detail/?cid=nrcs143_013696"
    tables = pd.read_html(url)
    
    df = tables[0][:-1].copy()
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    df.rename(columns={"postal_code": "alpha_code"}, inplace=True)
    df["fips"] = df["fips"].astype(int).astype(str).str.zfill(2)
    
    return df

In [ ]:
def get_usa_state_population(url=None, skiprows=3, skipfooter=7, cols_to_keep=None):
    """
    Returns a DataFrame of United States 2019 population estimates by state from the US Census website.
    
    Args:
        url (str): Link to the US Census' Excel file.
        skiprows (int): Rows to skip at the top of the file.
        skipfooter (int): Rows to skip at the end of the file.
        cols_to_keep (list): 0-indexed list of column indices to keep.
        
    Returns:
        pd.DataFrame
    """
    if url is None:
        url = r"https://www2.census.gov/programs-surveys/popest/tables/2010-2019/state/totals/nst-est2019-01.xlsx"
        
    if cols_to_keep is None:
        cols_to_keep = [0, -1]
        
    df = pd.read_excel(url, skiprows=skiprows, skipfooter=skipfooter)
    df = df.iloc[:, cols_to_keep].copy()
    df.columns = ["state", "population"]
    df = df[df["state"].str.startswith(".")].copy()
    df["state"] = df["state"].str[1:]
    
    return df

In [ ]:
def get_usa_county_codes(url=None, table_index=0):
    """
    Returns a DataFrame of United States FIPS county codes from the NRCS website.
    
    Args:
        url (str): URL to NRCS county FIPS codes page.
        table_index (int): Position of the table of interest within the page's HTML elements.
        
    Returns:
        pd.DataFrame
    """
    if url is None:
        url = r"https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697"
    tables = pd.read_html(url)
    
    df = tables[0][:-1].copy()
    df.columns = [c.strip().lower() for c in df.columns]
    df["fips"] = df["fips"].astype(int).astype(str).str.zfill(5)
    
    return df

In [ ]:
def get_usa_county_population(url=None, skiprows=3, skipfooter=6, cols_to_keep=None):
    """
    TBC
    """
    if url is None:
        url = r"https://www2.census.gov/programs-surveys/popest/tables/2010-2019/counties/totals/co-est2019-annres.xlsx"
        
    if cols_to_keep is None:
        cols_to_keep = [0, -1]
        
    df = pd.read_excel(url, skiprows=skiprows, skipfooter=skipfooter)
    df = df.iloc[:, cols_to_keep].copy()
    df.columns = ["county", "population"]
    df = df[df["county"].str.startswith(".")].copy()
    df["county"] = df["county"].str[1:]
    df["county_state"] = df["county"].str.replace(" County, ", " ")
    df["state"] = df["county_state"].str.split(" ").str[-1]
    df["county"] = df["county_state"].str.split(" ").str[0]
    df = df[["county_state", "state", "county", "population"]]
    
    return df

In [ ]:
get_usa_county_population()

In [ ]:
df_state_codes = get_usa_state_codes()
df_county_codes = get_usa_county_codes()
df_states = get_nytimes_state_data()
# df_states = df_states.merge(df_state_codes[["fips", "alpha_code"]], how="left", left_on="fips", right_on="fips")
df_counties = get_nytimes_county_data()

In [ ]:
df_state_codes.head()

In [ ]:
df_states.head()

In [ ]:
df_state_codes[df_state_codes["fips"] == "01"]["name"].iloc[0]

In [ ]:
set(df_state_codes["fips"]).difference(set(df_states["fips"]))

In [ ]:
geojson_url = r"https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
res = requests.get(geojson_url)
geojson = json.loads(res.content)

In [ ]:
fig = px.choropleth(df_counties, geojson=geojson_url, locations="fips", scope="usa", color=np.log10(df_counties["cum_cases"]), 
                    animation_frame="date", range_color=[0, np.log10(df_counties["cum_cases"]).max()], color_continuous_scale=px.colors.sequential.Reds, hover_name="county", hover_data=["cum_cases"])
fig.update_layout(coloraxis_colorbar=dict(title="Cumulative cases", thicknessmode="pixels", thickness=25, lenmode="pixels", len=397, yanchor="middle", y=.5, ticks="outside", 
                                          tickvals=list(range(math.ceil(np.log10(df_counties["cum_cases"].max())))),
                                          ticktext=[round(10 ** x, 0) for x in list(range(math.ceil(np.log10(df_counties["cum_cases"].max()))))]))
fig.show()

In [ ]:
fig = px.choropleth(df_states, locations="alpha_code", locationmode="USA-states", scope="usa", color=np.log10(df_states["cum_cases"]), range_color=[0, np.log10(df_states["cum_cases"]).max()], color_continuous_scale=px.colors.sequential.Reds,
                    hover_name="state", hover_data=["cum_cases"], animation_frame="date")
fig.update_layout(coloraxis_colorbar=dict(title="Cumulative cases", thicknessmode="pixels", thickness=25, lenmode="pixels", len=397, yanchor="middle", y=.5, ticks="outside", 
                                          tickvals=list(range(math.ceil(np.log10(df_states["cum_cases"].max())))),
                                          ticktext=[round(10 ** x, 0) for x in list(range(math.ceil(np.log10(df_states["cum_cases"].max()))))]))
fig.show()

In [ ]:
state_code = "02"
filtered_geojson = {"type": "FeatureCollection", "features": [d for d in geojson["features"] if d["properties"]["STATE"] == state_code]}
filtered_df = df_counties[df_counties["fips"].str[:2] == state_code].copy()

fig = px.choropleth(filtered_df, geojson=filtered_geojson, color="cum_cases", locations="fips", featureidkey="id", animation_frame="date")
fig.update_geos(fitbounds="locations")
pio.show(fig)

In [ ]:
filtered_df = df_counties[df_counties["fips"] == "01133"].copy()
filtered_df

In [ ]:
len(df_counties["fips"].unique())

In [ ]:
len(df_counties["county"].unique())

In [ ]:
df_counties.head()

In [ ]:
df_counties[df_counties["fips"].str[:2] == "01"].tail(50)

In [ ]:
filtered_geojson["features"][0]

In [ ]:
geojson

In [ ]:
geojson_test = px.data.election_geojson()
geojson_test